In [1]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

ModuleNotFoundError: No module named 'gensim'

In [2]:
!pip install gensim

  Using cached gensim-4.0.1-cp38-cp38-manylinux1_x86_64.whl (23.9 MB)
     |████████████████████████████████| 58 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 28.4 MB 728 kB/s eta 0:00:011


In [3]:
import os
import warnings
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument

/home/sunny/sunny_study/chatbot_test/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:

faqs = [["1", "당해년도 납입액은 수정 가능 한가요?", "네, 당해년도 납입액은 12464 화면 등록전까지 수정 가능합니다."],
            ["2", "대리인통보 대상계좌 기준은 어떻게 되나요?", "모계좌 기준 가장 최근에 개설된 계좌의 관리점에서 조회 됩니다.  의원폐쇄된 자계좌는 조회대상 계좌에서 제외됩니다. 계좌주 계좌가 사절원 계좌가 아닌 경우만 조회됩니다"],
            ["3", "등록가능 단말기수는 어떻게 되나요?", "5대까지 등록 가능입니다."],
            ["4", "모바일계좌개설 가능한 시간은 어떻게 되나요?", "08:00 ~ 20:00(영업일만 가능"],
            ["5", "미국인일때 미국납세자등록번호 작성 방법은 어떻게 되나요?", "계좌주가 미국인일 때 계좌주의 미국납세자등록번호(사회보장번호(Social Security Number), 고용주식별번호(Employer Identification Number), 개인납세자번호(Individual Taxpayer Identification Number))를 기재합니다.."]
    ]

In [5]:
faqs

[['1', '당해년도 납입액은 수정 가능 한가요?', '네, 당해년도 납입액은 12464 화면 등록전까지 수정 가능합니다.'],
 ['2',
  '대리인통보 대상계좌 기준은 어떻게 되나요?',
  '모계좌 기준 가장 최근에 개설된 계좌의 관리점에서 조회 됩니다.  의원폐쇄된 자계좌는 조회대상 계좌에서 제외됩니다. 계좌주 계좌가 사절원 계좌가 아닌 경우만 조회됩니다'],
 ['3', '등록가능 단말기수는 어떻게 되나요?', '5대까지 등록 가능입니다.'],
 ['4', '모바일계좌개설 가능한 시간은 어떻게 되나요?', '08:00 ~ 20:00(영업일만 가능'],
 ['5',
  '미국인일때 미국납세자등록번호 작성 방법은 어떻게 되나요?',
  '계좌주가 미국인일 때 계좌주의 미국납세자등록번호(사회보장번호(Social Security Number), 고용주식별번호(Employer Identification Number), 개인납세자번호(Individual Taxpayer Identification Number))를 기재합니다..']]

In [6]:
import jpype
from konlpy.tag import Kkma

In [9]:
kkma = Kkma()

def tokenize_kkma(doc):
    jpype.attachThreadToJVM()
    token_doc = ['/'.join(word) for word in kkma.pos(doc) ]
    return token_doc

In [8]:
tokenize_kkma(faqs[0][1])

/tmp/ipykernel_193242/1807617477.py:4: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  jpype.attachThreadToJVM()


['당해/NNG',
 '년도/NNM',
 '납입/NNG',
 '액/XSN',
 '은/JX',
 '수정/NNG',
 '가능/NNG',
 '한/MDN',
 '가요/NNG',
 '?/SF']

In [10]:

# 리스트에서 각 문장부분 토큰화
token_faqs = [(tokenize_kkma(row[1]), row[0]) for row in faqs]

# Doc2Vec에서 사용하는 태그문서형으로 변경
tagged_faqs = [TaggedDocument(d, [c]) for d, c in token_faqs]

In [11]:
tagged_faqs

[TaggedDocument(words=['당해/NNG', '년도/NNM', '납입/NNG', '액/XSN', '은/JX', '수정/NNG', '가능/NNG', '한/MDN', '가요/NNG', '?/SF'], tags=['1']),
 TaggedDocument(words=['대리인/NNG', '통보/NNG', '대상/NNG', '계좌/NNG', '기준/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['2']),
 TaggedDocument(words=['등록/NNG', '가능/NNG', '단말/NNG', '기수/NNG', '는/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['3']),
 TaggedDocument(words=['모바일/NNG', '계좌/NNG', '개설/NNG', '가능/NNG', '하/XSV', 'ㄴ/ETD', '시간/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['4']),
 TaggedDocument(words=['미국인/NNG', '일/NNG', '때/NNG', '미국/NNP', '납세자/NNG', '등록/NNG', '번호/NNG', '작성/NNG', '방법/NNG', '은/JX', '어떻/VA', '게/ECD', '되/VV', '나요/EFQ', '?/SF'], tags=['5'])]

In [29]:
# make model
import multiprocessing
cores = multiprocessing.cpu_count()
d2v_faqs = doc2vec.Doc2Vec(
                                vector_size=50, 
                                alpha=0.025,
                                min_alpha=0.025,
                                hs=1,
                                negative=0,
                                dm=0,
                                dbow_words = 1,
                                min_count = 1,
                                workers = cores,
                                seed=0
                                )
d2v_faqs.build_vocab(tagged_faqs)
# train document vectors
for epoch in range(10):
    d2v_faqs.train(tagged_faqs, total_examples = d2v_faqs.corpus_count, epochs = d2v_faqs.epochs)
    d2v_faqs.alpha -= 0.0025 # decrease the learning rate
    d2v_faqs.min_alpha = d2v_faqs.alpha # fix the learning rate, no decay

print(d2v_faqs.infer_vector(["당해년도 납입액은 수정 가능 한가요"]))

[ 8.9144632e-03 -1.2380857e-03 -7.7326046e-03  4.1886973e-03
  9.1657997e-04  7.8554936e-03 -7.3586390e-03  5.0172093e-04
 -3.3302857e-03 -2.7857542e-03  2.2979425e-03  2.1421539e-03
  3.0059421e-03 -7.1247935e-04 -2.6044173e-03  5.8888076e-03
 -3.4865898e-03 -7.0900319e-04  4.2798412e-03 -5.0855316e-03
 -9.9084796e-03 -5.7669622e-03  7.7680661e-03  9.6488632e-03
 -9.2756273e-03  2.4198056e-03 -6.6461116e-03 -4.1193920e-03
  1.2721396e-03  2.6708234e-03 -4.4824653e-03  4.2603421e-03
  4.3352844e-05 -9.0530398e-04 -6.7668520e-03 -2.8424023e-04
 -8.8278875e-03  9.1535570e-03 -9.8010781e-04 -2.1644890e-04
  1.0549069e-03 -7.0455801e-03 -1.8412108e-03 -9.5050344e-03
 -7.7899303e-03 -5.4676151e-03 -2.7949470e-03  9.7417207e-03
  9.1816699e-03  4.1070342e-04]


In [30]:
predict_vector = d2v_faqs.infer_vector(["당해년도 납입액은 수정 가능 한가요?"])

In [31]:
d2v_faqs.docvecs.most_similar([predict_vector], topn=2)


/tmp/ipykernel_193242/22855520.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  d2v_faqs.docvecs.most_similar([predict_vector], topn=2)


[('2', 0.08381495624780655), ('4', 0.07660745084285736)]

In [32]:

test_string = "대리인통보 대상계좌 기준은 어떻게 되나요?"
tokened_test_string = tokenize_kkma(test_string)
tokened_test_string

['대리인/NNG',
 '통보/NNG',
 '대상/NNG',
 '계좌/NNG',
 '기준/NNG',
 '은/JX',
 '어떻/VA',
 '게/ECD',
 '되/VV',
 '나요/EFQ',
 '?/SF']

In [33]:

test_vector = d2v_faqs.infer_vector(tokened_test_string)
d2v_faqs.docvecs.most_similar([test_vector], topn=2)

/tmp/ipykernel_193242/3448787366.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  d2v_faqs.docvecs.most_similar([test_vector], topn=2)


[('3', 0.08339903503656387), ('4', 0.04971155524253845)]